In [1]:
import os
import cv2
import numpy as np
import math
from sklearn.neighbors import KDTree
from sklearn.cluster import MiniBatchKMeans
import numpy as np
import copy


"""
构造KD树
"""
def get_KDtree(txt_path):
    data = np.loadtxt(txt_path)
    return KDTree(data)
    

"""
生成聚类结果
"""
def kd_tree(trans):
    h,w = trans.shape[0:2]
    tree = get_KDtree('TR1000.txt')
    trans = np.reshape(trans,(h*w,-1))
    dist,ind = tree.query(trans,k=1)
    labels = np.reshape(ind,(h,w))
    return labels

def guided_filter(I,p,win_size,eps):

    mean_I = cv2.blur(I,(win_size,win_size))
    mean_p = cv2.blur(p,(win_size,win_size))

    corr_I = cv2.blur(I*I,(win_size,win_size))
    corr_Ip = cv2.blur(I*p,(win_size,win_size))

    var_I = corr_I-mean_I*mean_I
    cov_Ip = corr_Ip - mean_I*mean_p

    a = cov_Ip/(var_I+eps)
    b = mean_p-a*mean_I

    mean_a = cv2.blur(a,(win_size,win_size))
    mean_b = cv2.blur(b,(win_size,win_size))

    q = mean_a*I + mean_b
    return q

"""
功能：透射率图初始估计
参数说明：
img:
"""
def tran_eas(img,dist_from_airlight,labels,A,K=1000):
    transmission_estimation = copy.deepcopy(dist_from_airlight)
    dist_from_airlight = np.abs(dist_from_airlight)
    trans = np.zeros_like(dist_from_airlight)
    for i in range(K):
        mask = np.where(labels==i)
        if mask[0].shape[0]>0:
            max_rad = max(dist_from_airlight[mask])
            transmission_estimation[mask] /= max_rad
    trans_min = 0.1
    transmission_estimation = np.minimum(np.maximum(transmission_estimation, trans_min),1)
    trans_lower_bound = 1-np.min(img/A,axis=2)
    transmission_estimation = np.maximum(transmission_estimation,trans_lower_bound)
#    print(img.dtype,transmission_estimation.dtype,img.shape,transmission_estimation.shape)
   # cv2.imshow('t1', transmission_estimation)
    #cv2.imwrite('./result_improved/haze_line_t1_'+str(767)+'.png', transmission_estimation*255)
    gimfiltR = 7 #引导滤波时半径的大小
    eps = 10**-3 #引导滤波时epsilon的值
    img = np.float32(img)
    gray_I = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    dst = guided_filter(gray_I/255.0,transmission_estimation,15,0.001)
    
    #guided_filter = Guided_Berman(img, gimfiltR, eps)
    #dst = guided_filter.filter(transmission_estimation)
#    dst = cv2.ximgproc.guidedFilter(guide=img, src=transmission_estimation, radius=16, eps=50, dDepth=-1)
  #  cv2.imshow('t2', dst)
    #cv2.imwrite('./result_improved/haze_line_t2_'+str(767)+'.png', dst*255)
    return dst
"""
参数解释：
img_path:为输入图像路径
save_path:为复原图像保存路径，默认为空
"""
def berman_dehaze(haze_img,save_path=None):
#    haze_img = cv2.imread(img_path)
    show_img = copy.deepcopy(haze_img)/float(255)
    #print('show_img',show_img.dtype)
    airlight = eas_Berman(haze_img, r=7, eps=0.001, w=0.95, maxV1=0.95)/255
    #airlight = 209.0/255
    #print(airlight)
    h,w = haze_img.shape[0:2]
    h,w,channel = haze_img.shape
    haze_img = haze_img.astype(np.float32)/255
#    haze_img = haze_img/255
    dist_from_airlight = np.zeros_like(haze_img)
    radius = np.zeros((h,w),dtype = np.float64)
    for id in range(channel):
        dist_from_airlight[:,:,id] = haze_img[:,:,id]-airlight
        radius += np.power(dist_from_airlight[:,:,id],2)
    radius = np.sqrt(radius+0.000001)
    I_A = copy.deepcopy(radius)
    radius = cv2.merge((radius,radius,radius))
    points = dist_from_airlight/radius
    labels = kd_tree(points)
    
    t = tran_eas(show_img,I_A,labels,airlight)
    t = cv2.merge((t,t,t))
    t = t.astype(np.float32)
    J = (dist_from_airlight/t+airlight)*255
#    J *= 255
    J = np.clip(J,0,255)
    J = J.astype(np.uint8)
#    print((J*255).dtype)
#    J = (255*J).astype(np.uint8)
#    print(save_path)
#    while True:
#        cv2.imshow('J',J)
#        if cv2.waitKey(10) == 27:
#            cv2.destroyAllWindows()
#            break
    if save_path:
        cv2.imwrite(save_path,J*255)
    return J


In [2]:
class Guided_Berman:
    
    def __init__(self, I, radius=5, epsilon=0.4):

        self._radius = 2 * radius + 1
        self._epsilon = epsilon
        self._I = self._toFloatImg(I)
        
        self._initFilter()

    def _toFloatImg(self, img):
        if img.dtype == np.float32:
            return img
        return ( 1.0 / 255.0 ) * np.float32(img)

    def _initFilter(self):
        I = self._I
        r = self._radius
        eps = self._epsilon

        Ir, Ig, Ib = I[:, :, 0], I[:, :, 1], I[:, :, 2]

        self._Ir_mean = cv2.blur(Ir, (r, r))
        self._Ig_mean = cv2.blur(Ig, (r, r))
        self._Ib_mean = cv2.blur(Ib, (r, r))

        Irr_var = cv2.blur(Ir ** 2, (r, r)) - self._Ir_mean ** 2 + eps                                       
        Irg_var = cv2.blur(Ir * Ig, (r, r)) - self._Ir_mean * self._Ig_mean                                  
        Irb_var = cv2.blur(Ir * Ib, (r, r)) - self._Ir_mean * self._Ib_mean                                  
        Igg_var = cv2.blur(Ig * Ig, (r, r)) - self._Ig_mean * self._Ig_mean + eps                            
        Igb_var = cv2.blur(Ig * Ib, (r, r)) - self._Ig_mean * self._Ib_mean                                  
        Ibb_var = cv2.blur(Ib * Ib, (r, r)) - self._Ib_mean * self._Ib_mean + eps                                                                                     


        self._Ir_mean = cv2.blur(Ir, (r, r))
        self._Ig_mean = cv2.blur(Ig, (r, r))
        self._Ib_mean = cv2.blur(Ib, (r, r))

        Irr_var = cv2.blur(Ir ** 2, (r, r)) - self._Ir_mean ** 2 + eps                                       
        Irg_var = cv2.blur(Ir * Ig, (r, r)) - self._Ir_mean * self._Ig_mean                                  
        Irb_var = cv2.blur(Ir * Ib, (r, r)) - self._Ir_mean * self._Ib_mean                                  
        Igg_var = cv2.blur(Ig * Ig, (r, r)) - self._Ig_mean * self._Ig_mean + eps                            
        Igb_var = cv2.blur(Ig * Ib, (r, r)) - self._Ig_mean * self._Ib_mean                                  
        Ibb_var = cv2.blur(Ib * Ib, (r, r)) - self._Ib_mean * self._Ib_mean + eps                                                       


        Irr_inv = Igg_var * Ibb_var - Igb_var * Igb_var                                                      
        Irg_inv = Igb_var * Irb_var - Irg_var * Ibb_var                                                      
        Irb_inv = Irg_var * Igb_var - Igg_var * Irb_var                                                      
        Igg_inv = Irr_var * Ibb_var - Irb_var * Irb_var                                                      
        Igb_inv = Irb_var * Irg_var - Irr_var * Igb_var                                                      
        Ibb_inv = Irr_var * Igg_var - Irg_var * Irg_var                                                      
        
        I_cov = Irr_inv * Irr_var + Irg_inv * Irg_var + Irb_inv * Irb_var                                    
        Irr_inv /= I_cov                                                                                     
        Irg_inv /= I_cov                                                                                     
        Irb_inv /= I_cov                                                                                     
        Igg_inv /= I_cov                                                                                     
        Igb_inv /= I_cov                                                                                     
        Ibb_inv /= I_cov                                                                                     
        
        self._Irr_inv = Irr_inv                                                                              
        self._Irg_inv = Irg_inv                                                                              
        self._Irb_inv = Irb_inv                                                                              
        self._Igg_inv = Igg_inv                                                                              
        self._Igb_inv = Igb_inv                                                                              
        self._Ibb_inv = Ibb_inv                  

    def _computeCoefficients(self, p):
        r = self._radius                                                             
        I = self._I                                                                 
        Ir, Ig, Ib = I[:, :, 0], I[:, :, 1], I[:, :, 2]                                                          
        

        p_mean = cv2.blur(p, (r, r))                             
        Ipr_mean = cv2.blur(Ir * p, (r, r))                                                         
        Ipg_mean = cv2.blur(Ig * p, (r, r))                                                    
        Ipb_mean = cv2.blur(Ib * p, (r, r))             



        Ipr_cov = Ipr_mean - self._Ir_mean * p_mean                                                 
        Ipg_cov = Ipg_mean - self._Ig_mean * p_mean                                                     
        Ipb_cov = Ipb_mean - self._Ib_mean * p_mean                                                       
                                                                                                                 
        ar = self._Irr_inv * Ipr_cov + self._Irg_inv * Ipg_cov + self._Irb_inv * Ipb_cov                 
        ag = self._Irg_inv * Ipr_cov + self._Igg_inv * Ipg_cov + self._Igb_inv * Ipb_cov                
        ab = self._Irb_inv * Ipr_cov + self._Igb_inv * Ipg_cov + self._Ibb_inv * Ipb_cov    

        b = p_mean - ar * self._Ir_mean - ag * self._Ig_mean - ab * self._Ib_mean                                                                                                                                         

        ar_mean = cv2.blur(ar, (r, r))          
        ag_mean = cv2.blur(ag, (r, r))                                                                   
        ab_mean = cv2.blur(ab, (r, r))                                                                      
        b_mean = cv2.blur(b, (r, r))                                                                                                                                              

        return ar_mean, ag_mean, ab_mean, b_mean            

    def _computeOutput(self, ab, I):
    
        ar_mean, ag_mean, ab_mean, b_mean = ab
        Ir, Ig, Ib = I[:, :, 0], I[:, :, 1], I[:, :, 2]
        q = ar_mean * Ir + ag_mean * Ig + ab_mean * Ib + b_mean
        return q

    def filter(self, p):
        
        p_32F = self._toFloatImg(p)
        ab = self._computeCoefficients(p)
        return self._computeOutput(ab, self._I)


In [3]:
import cv2
import numpy as np

def MinFilter_berman(src, r=7):
    '''最小值滤波，r是滤波器半径'''
    return cv2.erode(src, np.ones((2 * r + 1, 2 * r + 1)))


def eas_Berman(m, r, eps, w, maxV1):                 # 输入rgb图像，值范围[0,1]
    '''计算大气遮罩图像V1和光照值A, V1 = 1-t/A'''
    V1 = np.min(m, 2)  
    Dark_Channel = MinFilter_berman(V1, r)
    bins = 2000
    ht = np.histogram(Dark_Channel, bins)                  # 计算大气光照A
    d = np.cumsum(ht[0]) / float(Dark_Channel.size)
    for lmax in range(bins - 1, 0, -1):
        if d[lmax] <= 0.999:
            break
    A = np.mean(m, 2)[Dark_Channel >= ht[1][lmax]].max()
#    V1 = np.minimum(V1 * w, maxV1)               # 对值范围进行限制
#    print(A)
    return A

In [4]:
def berman_dehaze(haze_img,save_path=None):
#    haze_img = cv2.imread(img_path)
    show_img = copy.deepcopy(haze_img)/float(255)

    #airlight = eas_Berman(haze_img, r=7, eps=0.001, w=0.95, maxV1=0.95)/255
    A = 255
    airlight = A/255.0

    h,w = haze_img.shape[0:2]
    h,w,channel = haze_img.shape
    haze_img = haze_img.astype(np.float32)/255
#    haze_img = haze_img/255
    dist_from_airlight = np.zeros_like(haze_img)
    radius = np.zeros((h,w),dtype = np.float64)
    for id in range(channel):
        dist_from_airlight[:,:,id] = haze_img[:,:,id]-airlight
        radius += np.power(dist_from_airlight[:,:,id],2)
    radius = np.sqrt(radius+0.000001)
    I_A = copy.deepcopy(radius)
    radius = cv2.merge((radius,radius,radius))
    points = dist_from_airlight/radius
    labels = kd_tree(points)
    
    t = tran_eas(show_img,I_A,labels,airlight)
    t = cv2.merge((t,t,t))
    t = t.astype(np.float32)
    J = (dist_from_airlight/t+airlight)*255
#    J *= 255
    J = np.clip(J,0,255)
    J = J.astype(np.uint8)
#    print((J*255).dtype)
#    J = (255*J).astype(np.uint8)
#    print(save_path)
#    while True:
#        cv2.imshow('J',J)
#        if cv2.waitKey(10) == 27:
#            cv2.destroyAllWindows()
#            break
    if save_path:
        cv2.imwrite(save_path,J*255)
    return J

In [61]:
import time
i = 231
img = cv2.imread('./images/'+str(i) + '.png')
time1 = time.time()
dehazed_img = berman_dehaze(img)
time2 = time.time()
print("time:"+str(time2-time1))
bGamma = False
if(bGamma):
    J = dehazed_img
    J = J / 255.0
    J = J ** (np.log(0.5) / np.log(J.mean()))  # gamma校正
    J = J * 255.0
    dehazed_img = J
dehazed_img = np.clip(dehazed_img,0,255)
show_img = dehazed_img.astype(np.uint8)
cv2.imwrite('./result_improved/haze_line_J_'+str(i)+'.png', show_img)
cv2.imshow('J', show_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

time:4.392440319061279
